<a href="https://colab.research.google.com/github/disha-cpu/Resources-For-Stat-Data/blob/master/4D_DBSCAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing libraries
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from plotnine import *
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
import plotly.express as px
import random
import math
%matplotlib inline

In [ ]:
# reading data
data = pd.read_csv('/content/weatherGOA.csv')
# formatting time
data = data.replace(to_replace ="24:00:00", value ="00:00")
data['time'] = data['Date'] +" "+ data['UT time'] 
data['time'] = pd.to_datetime(data['time'])
# converting temprature from K to C
data = data[data['time'].dt.minute == 0]
data['Temperature (C)'] = data['Temperature (K)'] - 273.15
data['Hour'] = data['time'].dt.hour.astype(float)
# 4D dataset
dataset = data[['Hour',"Temperature (C)","Relative Humidity (%)", "Pressure (hPa)"]]
dataset

# from dataset
# Temprature - 23 to 32
# Pressure - 991 to 1000
# Humidity - 70 to 90

# from site
# Temp - 18 to 32
# Pressure - 1007 to 1013
# Humidity - 64 to 90

,Hour,Temperature (C),Relative Humidity (%),Pressure (hPa)
0,0.0,26.35,88.12,991.03
4,1.0,26.39,87.56,991.38
8,2.0,26.66,86.70,991.93
12,3.0,27.00,85.48,992.40
16,4.0,27.32,84.08,992.84
...,...,...,...,...
108076,19.0,26.38,87.77,995.67
108080,20.0,26.30,87.67,994.88
108084,21.0,26.25,87.40,994.18
108088,22.0,26.24,87.11,993.80


In [ ]:
import statistics
statistics.mean(dataset['Temperature (C)'])

27.13796625222027

In [ ]:
statistics.mean(dataset['Relative Humidity (%)'])

70.92891614860865

In [ ]:
statistics.mean(dataset['Pressure (hPa)'])

995.7055987270575

In [ ]:
import collections
collections.Counter(round(dataset['Temperature (C)']))

Counter({19.0: 7,
         20.0: 52,
         21.0: 209,
         22.0: 528,
         23.0: 983,
         24.0: 1582,
         25.0: 3284,
         26.0: 4971,
         27.0: 5058,
         28.0: 3701,
         29.0: 1947,
         30.0: 1542,
         31.0: 1232,
         32.0: 1011,
         33.0: 559,
         34.0: 278,
         35.0: 70,
         36.0: 10})

In [ ]:
collections.Counter(round(dataset['Relative Humidity (%)']))

Counter({19.0: 1,
         20.0: 1,
         21.0: 4,
         22.0: 4,
         23.0: 9,
         24.0: 18,
         25.0: 27,
         26.0: 30,
         27.0: 24,
         28.0: 50,
         29.0: 51,
         30.0: 66,
         31.0: 65,
         32.0: 94,
         33.0: 89,
         34.0: 128,
         35.0: 142,
         36.0: 154,
         37.0: 161,
         38.0: 166,
         39.0: 176,
         40.0: 192,
         41.0: 220,
         42.0: 212,
         43.0: 190,
         44.0: 203,
         45.0: 221,
         46.0: 230,
         47.0: 222,
         48.0: 256,
         49.0: 242,
         50.0: 284,
         51.0: 256,
         52.0: 273,
         53.0: 314,
         54.0: 322,
         55.0: 320,
         56.0: 332,
         57.0: 342,
         58.0: 337,
         59.0: 347,
         60.0: 377,
         61.0: 372,
         62.0: 352,
         63.0: 353,
         64.0: 392,
         65.0: 386,
         66.0: 398,
         67.0: 418,
         68.0: 401,
         69.0: 431,


In [ ]:
collections.Counter(round(dataset['Pressure (hPa)']))

Counter({987.0: 14,
         988.0: 49,
         989.0: 129,
         990.0: 356,
         991.0: 882,
         992.0: 1849,
         993.0: 2672,
         994.0: 3351,
         995.0: 3644,
         996.0: 3727,
         997.0: 3287,
         998.0: 2751,
         999.0: 1984,
         1000.0: 1257,
         1001.0: 617,
         1002.0: 323,
         1003.0: 88,
         1004.0: 32,
         1005.0: 9,
         1006.0: 3})

In [ ]:
# plotting graph
dummy = dataset.copy()
px.scatter_3d(dummy, x='Relative Humidity (%)', y='Temperature (C)', z='Pressure (hPa)', color = dummy['Hour']).update_traces(marker={'size': 2})


In [ ]:
d1 = dummy.copy()
d1[['Hour',"Temperature (C)","Relative Humidity (%)", "Pressure (hPa)"]] = StandardScaler().fit_transform(d1.values)

In [ ]:
# getting epsilon value
df = dummy
mins = len(df)//1000
nn = NearestNeighbors(n_neighbors=mins+1)
nn.fit(d1[['Hour',"Temperature (C)","Relative Humidity (%)", "Pressure (hPa)"]])
distances, neighbors = nn.kneighbors(d1[['Hour',"Temperature (C)","Relative Humidity (%)", "Pressure (hPa)"]])

# sort the distances
distances = np.sort(distances[:, mins], axis = 0)

#plot the distances
distances_df = pd.DataFrame({"distances": distances,
                             "index": list(range(0,len(distances)))})

px.line(distances_df, x='index', y='distances')

def calc_distance(x1, y1, a, b, c):
  d = abs((a * x1 + b * y1 + c)) / (math.sqrt(a * a + b * b))
  return d

def find_eps():
  # (y1 – y2)x + (x2 – x1)y + (x1y2 – x2y1) = 0
  a = distances[0] - distances[-1]  #y
  b = distances_df.index[-1] - distances_df.index[0]    #x
  c1 = distances_df.index[0] * distances[-1]
  c2 = distances_df.index[-1] * distances[0]
  c = c1 - c2

  distance_of_points_from_line = []
  for k in range(len(distances_df)):
    distance_of_points_from_line.append(
        calc_distance(distances_df.index[k], distances[k], a, b, c))
    
  dist = pd.Series(distance_of_points_from_line)
  index_max = dist.idxmax()
  return distances[index_max]

In [ ]:
# applying dbscan
db1 = DBSCAN(eps =  find_eps(), min_samples = mins).fit(d1)
labsList = ["Noise"]
labsList = labsList  + ["Cluster " + str(i) for i in range(1,len(set(db1.labels_)))]
d1["assignments"] = db1.labels_
color = d1["assignments"].astype(int)
n_noise_ = list(db1.labels_).count(-1)
print("Estimated number of noise points: %d" % n_noise_)

Estimated number of noise points: 395


In [ ]:
# plotting graph with anomalies
px.scatter_3d(dummy, x='Relative Humidity (%)', y='Temperature (C)', z='Pressure (hPa)', color = color).update_traces(marker={'size': dummy['Hour']})

In [ ]:
df1 = data.copy()

In [ ]:
# anomalous data
anomaly = d1[d1.assignments == -1]
anomalies_index = list(anomaly.index)
temp_list = []
time_list = []
humidity_list = []
pressure_list = []
# ["Hour","Temperature (C)","Humidity", "Pressure (millibars)"]
for index in anomalies_index:
  temp = df1.loc[index]['Temperature (C)']
  temp_list.append(temp)
  time = df1.loc[index]['time']
  time_list.append(time)
  humidity = df1.loc[index]['Relative Humidity (%)']
  humidity_list.append(humidity)
  pressure = df1.loc[index]['Pressure (hPa)']
  pressure_list.append(pressure)
dic = {
    'Time': time_list,
    'Temperature': temp_list,
    'Humidity': humidity_list,
    'Pressure': pressure_list
}

anomaly_df = pd.DataFrame(dic)
anomaly_df

,Time,Temperature,Humidity,Pressure
0,2016-10-19 00:00:00,22.32,74.45,993.82
1,2016-10-20 22:00:00,22.68,86.86,993.30
2,2016-10-20 23:00:00,22.18,87.33,993.58
3,2016-10-20 00:00:00,21.77,87.80,994.27
4,2016-10-21 01:00:00,21.54,87.91,995.14
...,...,...,...,...
390,2019-07-21 08:00:00,27.91,87.23,987.75
391,2019-07-21 09:00:00,27.74,88.08,987.24
392,2019-07-21 10:00:00,27.56,88.96,986.97
393,2019-07-21 11:00:00,27.44,89.53,986.99


In [ ]:
dummy['assignments']=d1['assignments']

In [ ]:
from sklearn import metrics
metrics.silhouette_score(dummy, dummy['assignments'])

0.2111148700767415